In [1]:
# !pip install transformers elasticsearch 

import numpy as np 
from transformers import AutoTokenizer, AutoModel 
from elasticsearch import Elasticsearch 
import torch 

# 인증정보를 사용해 일래스틱서치 접속 정보 정의
es = Elasticsearch(
    ['https://ea9fe54e452f481793fff7cbe9f3a720.us-central1.gcp.cloud.es.io:443'],
    http_auth=('elastic', 'Hg89Dmhkcmv9i5IP7FnPOCUm'),
    verify_certs=False
)
 

/Users/summerlee/.pyenv/versions/3.11.11/envs/jupyter/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://ea9fe54e452f481793fff7cbe9f3a720.us-central1.gcp.cloud.es.io:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_81809/3401737187.py:9: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [2]:

# dense vector 필드를 위한 매핑 정의
mapping = {
    'properties': {
        'embedding': {
            'type': 'dense_vector',
            'dims': 768   # dense vector의 차원수
        }
    }
}

# 정의한 매핑으로 인덱스 생성
es.indices.create(index='chapter-2', body={'mappings': mapping})

# 색인 할 문서 데이터 세트 구성
docs = [
    {
        'title': 'Document 1',
        'text': 'This is the first document.'
    },
    {
        'title': 'Document 2',
        'text': 'This is the second document.'
    },
    {
        'title': 'Document 3',
        'text': 'This is the third document.'
    }
]




/Users/summerlee/.pyenv/versions/3.11.11/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea9fe54e452f481793fff7cbe9f3a720.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
# BERT 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# BERT를 활용하여 각 문서의 임베딩 생성
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
    doc['embedding'] = output.tolist()

# 일래스틱서치에 문서 색인
for doc in docs:
    es.index(index='chapter-2', body=doc)

/Users/summerlee/.pyenv/versions/3.11.11/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea9fe54e452f481793fff7cbe9f3a720.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/summerlee/.pyenv/versions/3.11.11/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea9fe54e452f481793fff7cbe9f3a720.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/summerlee/.pyenv/versions/3.11.11/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea9fe54e452f481